In [28]:
from bm_preproc import BoyerMoore
from bm_preproc import z_array

In [45]:
def get_kmer_hits (p, p_bm, t):
    char_comp= 0
    align_try= 0
    kmer_hits= []
    t_idx= 0
    while t_idx <= len(t)-len(p)+1:
        align_try += 1
        #print('t_idx %i' %t_idx)
        for p_idx in range(len(p)-1,-1,-1):
            char_comp += 1
            if t[t_idx+p_idx] != p[p_idx]:
                bc_skip= p_bm.bad_character_rule(i=p_idx, c=t[t_idx+p_idx])
                gs_skip= p_bm.good_suffix_rule(p_idx)
                shift= max(bc_skip, gs_skip)
                #print('  Mismatch t_idx: %i, p_idx: %i' % (t_idx, p_idx))
                #print('shift %i' %shift)
                break
        if t[t_idx:t_idx+len(p)]== p:
            #print('Match %i' % t_idx)
            kmer_hits.append(t_idx)
            shift= p_bm.match_skip()
        t_idx += shift
    return [kmer_hits, char_comp, align_try]

pat= 'ACT'
query= 'ACTG'
tex= 'TCACTGACT' #'TCACTG'
bmObj= BoyerMoore(p= pat)
[kmer_hits, char_comp, align_try]= get_kmer_hits(pat, bmObj, tex)
print('Index hits:', kmer_hits)
print('Character comparisons: %i' %char_comp)
print('Alignments tried: %i' %align_try)

Index hits: [2, 6]
Character comparisons: 8
Alignments tried: 4


In [30]:
def verify(kmer_hits, t, query, offset=0):
    #offset: how far patern it to the right of query start
    matchPos= []
    for ih in kmer_hits:
        #print('index-hit: %i' %ih)
        if t[(ih-offset):(ih+len(query)-offset)] == query:
            #print('Check match %s vs query %s' % (t[(ih-offset):(ih+len(query)-offset)], query))
            matchPos.append(ih-offset)
    return matchPos

print( verify(kmer_hits, tex, query) )

[2]


In [31]:
def naive_exact(p,t):
    char_comp= 0
    align_try= 0
    matches= []
    for t_idx in range(len(t)-len(p)+1):
        align_try += 1
        #print('t index: %i' %t_idx)
        match= 1
        for p_idx in range(len(p)):
            char_comp += 1
            #print(' p index: %i' %p_idx)
            if t[t_idx+p_idx] != p[p_idx]:
                match= 0
                break
        if match == 1:
            #print('Match: %i' %t_idx)
            matches.append(t_idx)
    return([matches, char_comp, align_try])

[naiveExact_matches, char_comp, align_try]= naive_exact(query, tex)
print('Matches:', naiveExact_matches )
print('Character comparisons: %i' %char_comp)
print('Alignments tried: %i' %align_try)

Matches: [2]
Character comparisons: 9
Alignments tried: 6


In [64]:
genome= ''
f= open('chr1.GRCh38.excerpt.fasta')
while True:
    l= f.readline()
    l= l.rstrip()
    if len(l) == 0:
        break
    if l[0] != '>':
        genome += l

print( genome[:100] )
print('length %i' %len(genome))

TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAA
length 800000


In [33]:
# 1,
query= 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
[naiveExact_matches, char_comp, align_try]= naive_exact(query, genome)
print('Alignments %i' %align_try)

#2,
print('Character comparisons %i' %char_comp)

Alignments 799954
Character comparisons 984143


In [35]:
#3,
pat= 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'
tex= genome
bmObj= BoyerMoore(pat)
[kmer_hits, char_comp, align_try]= get_kmer_hits(pat, bmObj, tex)
print('kmer hits:', kmer_hits)
print('Character comparisons: %i' %char_comp)
print('Alignments tried: %i' %align_try)

Match 56922
kmer hits: [56922]
Character comparisons: 165191
Alignments tried: 127974


In [72]:
# 4+5, Pigeonhole
 #pattern has length 24
 #allow two MM

from kmer_index import Index

pat=    'GGCGCGGTGGCTCACGCCTGTAAT'   #'GGCGCGGTGTCTCACGCCTTTAAT' # #'GGCGCGGTGMCTCACGCCTMMAAT'
#genome= 'AAAGGCGCGGTGGCTCACGCCTGTAATAAA'
iObj= Index(genome, 8)
print(genome)

frag_cuts= range(0,25,8)
match_pos= set()
total_fragHits= 0
for i in range(1,len(frag_cuts)):
    print('\ni: %i' %i)
    frag= pat[frag_cuts[i-1]:frag_cuts[i]]
    print('frag: %s' %frag)
    
    #fragMatches= iObj.query(frag) #matches of fragment
    #### Boyer Moore check ###
    p_bm= BoyerMoore(frag)   #for Boyer-Moore check
    [fragMatches, char_comp, align_try]= get_kmer_hits (frag, p_bm, genome)
     #no verification required, because kmer == full fragment
    ##########################
    total_fragHits += len(fragMatches)
    print('Fragement matches:', fragMatches)
    if( len(fragMatches)== 0):
        continue

    for fM_no in fragMatches:
        mm_count= 0
        left_end= fM_no - (i-1)*8
        #print('  Left end: %i' %left_end)
        if left_end < 0 or left_end+len(pat)>len(genome):
            continue
        # look left
        for j in range(i*8):
            if genome[left_end+j] != pat[j]:
                mm_count += 1
            if mm_count > 2:
                break
        #print('  I looked left: %i mismatches' %mm_count)
        #look right
        #right_start= fM_no+8
        for j in range(i*8,len(pat)): #matched pos is not checked again
            if genome[left_end+j] !=  pat[j]:
                mm_count += 1
            if mm_count > 2:
                break
        #print('  I looked left and right: %i mismatches' %mm_count)
        if mm_count <= 2:
            match_pos.add(fM_no - (i-1)*8)

print("\nFull match positions:", sorted(match_pos))
print('%i full matches found' % len(match_pos))
print('%i total fragment hits found' % total_fragHits)

...

i: 1
frag: GGCGCGGT
Fragement matches: [56922, 57056, 83720, 84641, 147558, 160729, 191452, 262042, 364263, 657496, 681737, 717706, 725061]

i: 2
frag: GGCTCACG
Fragement matches: [18878, 56930, 160170, 262050, 273677, 282012, 364271, 421229, 429307, 465655, 472642, 489446, 551142, 621370, 657504, 717714, 724935]

i: 3
frag: CCTGTAAT
Fragement matches: [18749, 19182, 22413, 22548, 23019, 23154, 43143, 56938, 67379, 83736, 83879, 84657, 84791, 108126, 129010, 147574, 160745, 175326, 186012, 187671, 191468, 205397, 251106, 251240, 262058, 273685, 282020, 322751, 364279, 364412, 421237, 429315, 454348, 465663, 471982, 480517, 480658, 523101, 551150, 551843, 572212, 588494, 595557, 613475, 621507, 632321, 635947, 646504, 651539, 657512, 674072, 681753, 707167, 717722, 719434, 724943, 746636, 747375, 747511, 760505]

Full match positions: [56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359

In [71]:
print( pat )
print( genome[472634:472634+len(pat)] )

GGCGCGGTGGCTCACGCCTGTAAT
GGTGCGGTGGCTCACGTCTGTAAC


In [68]:
def naive_2mm(pattern, text):
    #naive pattern match, allowing two mismatches
    matchPosL=[]
    for i in range(len(text) - len(pattern) + 1):
        MM= 0
        #print('i: %i' %i)
        for j in range(len(pattern)):
            #print('\tj: %i' %j)
            if text[i+j] != pattern[j]:
                MM += 1
            if MM > 2:
                break
        if MM<= 2:
            matchPosL.append(i)
            #print('Match! (MM: %i)' %MM)
    return(matchPosL)

matchPos_naive= naive_2mm(pat, genome)
print( matchPos_naive )
print('%i full matches found' % len(matchPos_naive))

[56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 724927, 747359]
19 full matches found


In [67]:
#6, Pigeonhole with subsequence
 #pattern has length 24
 #allow two MM

from subseq_idx import SubseqIndex

pat=    'GGCGCGGTGGCTCACGCCTGTAAT'   #'GGCGCGGTGTCTCACGCCTTTAAT' # #'GGCGCGGTGMCTCACGCCTMMAAT'
#genome= 'AAAGGCGCGGTGGCTCACGCCTGTAATAAA'
k= 8 #length of subsequence
ival= 3 #every how manyth base
subseq_iObj= SubseqIndex(t= genome, k=k, ival=ival)
#print(genome)

match_pos= set()
total_fragHits= 0
for i in range(ival):
    #print('\nfrom: %i to %i, by %i' % (i,i+k*ival+1, ival))
    print('pat start %i' %i)
    end= k*ival-(ival-i)+1
    print('pat end %i' % end)
    frag= pat[i:end]
    print('frag: %s' %frag)
    
    fragMatches= subseq_iObj.query(frag) #matches of fragment
    total_fragHits += len(fragMatches)
    print('Fragement matches:', fragMatches)
    if( len(fragMatches)== 0):
        continue

    for fM_no in fragMatches:
        mm_count= 0
        left_end= fM_no - i
        #print('  Left end: %i' %left_end)
        if left_end < 0 or left_end+len(pat)>len(genome):
            continue
        # look verify
        for j in range(len(pat)):
            if genome[left_end+j] != pat[j]:
                mm_count += 1
            if mm_count > 2:
                break
        if mm_count <= 2:
            match_pos.add(fM_no - i)

print("\nFull match positions:", sorted(match_pos))
print('%i full matches found' % len(match_pos))
print('%i total fragment hits found' % total_fragHits)

pat start 0
pat end 22
frag: GGCGCGGTGGCTCACGCCTGTA
Fragement matches: [56922, 67486, 83863, 84641, 84775, 124024, 147558, 191452, 199607, 262042, 262174, 273669, 322735, 364263, 421354, 454332, 465647, 471966, 472634, 489019, 558456, 579737, 596898, 635931, 651523, 657496, 658702, 681737, 707151, 712449, 717706, 719418, 719557, 746620, 747359]
pat start 1
pat end 23
frag: GCGCGGTGGCTCACGCCTGTAA
Fragement matches: [22398, 32640, 56923, 84642, 100012, 108111, 137575, 147559, 151719, 160163, 160730, 262043, 273670, 364264, 366819, 421222, 429300, 465648, 479031, 551135, 551828, 635932, 657497, 681738, 717707, 724928, 745641, 783347, 794643]
pat start 2
pat end 24
frag: CGCGGTGGCTCACGCCTGTAAT
Fragement matches: [23005, 56924, 141048, 160731, 191454, 262044, 349191, 364265, 429301, 657498, 704733, 717708, 724929, 747361, 766421]

Full match positions: [56922, 84641, 147558, 160162, 160729, 191452, 262042, 273669, 364263, 421221, 429299, 465647, 551134, 635931, 657496, 681737, 717706, 72492